# 纠缠蒸馏 -- LOCCNet 设计协议

<em> Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved. </em>

## 概述

量子纠缠在量子通信，量子计算和其他许多量子技术中起着至关重要的作用。因此，如果我们想在这些领域构建实际应用，探测、传输和分发量子纠缠是必不可少的任务。但是在实际操作中，误差是不可避免的。这些误差可能来自于生产纠缠设备的缺陷，也可能是传输量子纠缠时的信道噪声。随着传输距离的增加，噪声会导致纠缠资源的不断减少。而纠缠蒸馏（entanglement distillation）这项技术的开发目的正是为了补偿各种噪声引起的纠缠资源消耗。基本的实现原理为通过操作多对含噪的纠缠态将纠缠资源集中在其中一对纠缠态上，并使之尽可能的接近**最大纠缠态**（qubit 情况下，通常是大家熟知的贝尔态）。从这个意义上讲，也可以将纠缠蒸馏看作一种提纯/纠错协议。此过程通常由 Alice 和 Bob 两个远程参与方执行，由于双方在空间上会相隔一定的距离，因此仅允许本地操作和经典通信（LOCC）[1]。纠缠蒸馏的概念最早由 Bennett 等人于 1996 年提出 [2]，最初的协议被称为 **BBPSSW 协议**。后来 Deutsch 等人提出了一个新的蒸馏协议，即 DEJMPS 协议 [3]。

但是，BBPSSW 协议和 DEJMPS 协议都是针对特定噪声态设计的，比如：BBPSSW 是针对 isotropic 态设计而 DEJMPS 协议是针对贝尔对角态（Bell-diagonal state）设计的。实际上，设计出一个通用的能够应对所有噪声进行提纯的协议几乎是不可能的。同时，由于 LOCC 协议结构的复杂性，每次遇到新种类的噪声时用纸和笔重新设计一个蒸馏协议是非常困难的。LOCCNet 作为一个设计 LOCC 协议的机器学习框架，就是为了解决上述问题而存在的。在 LOCCNet 的加持下，只要给定纠缠态中噪声的数学形式，设计对应的蒸馏方案就会变成一件十分简单的事情。

### 预备知识

在谈论纠缠蒸馏的时候，我们通常使用蒸馏后的输出态 $\rho_{out}$ 和贝尔态 $|\Phi^+\rangle$ 之间的**保真度** $F$ 来量化纠缠蒸馏协议的好坏，保真度 $F$ 定义为

$$
F(\rho_{out}, \Phi^+) \equiv \langle \Phi^+|\rho_{out}|\Phi^+\rangle.
\tag{1}
$$

**注意：** 通常情况下，LOCC 纠缠蒸馏协议并不是在所有测量结果下都算成功，一般记成功概率为 $p_{succ}$。

### 协议的设计逻辑

在本教程中，我们用4份相同的 **isotropic 态** （也被称为 Werner 态）蒸馏出一份具有更高保真度输出态（相较于初始态，输出态更接近贝尔态 $|\Phi^+\rangle$）。这种协议被称为 $4\rightarrow 1$ LOCC 蒸馏协议。相对应的，BBPSSW 和 DEJMPS 协议属于 $2\rightarrow 1$ LOCC 蒸馏协议（因为一开始使用了2份相同的初始态）。
Isotropic 态是由 $|\Phi^+\rangle$ 和完全混合态（白噪声）$I/4$ 组成,

$$
\rho_{\text{iso}}(p) = p\lvert\Phi^+\rangle \langle\Phi^+\rvert + (1-p)\frac{I}{4}, \quad p \in [0,1]
\tag{2}
$$

在我们的例子中 $p=0.7$，于是输入态就是

$$
\rho_{in} = \rho_{\text{iso}}(0.7)= 0.7\lvert\Phi^+\rangle \langle\Phi^+\rvert + 0.075 I.
\tag{3}
$$

为了通过 LOCC 完成纠缠蒸馏，我们需要两位身处于两个地方的参与者 $A$ (Alice) 和 $B$ (Bob)。在最开始的时候，他们共享四份纠缠量子比特对 $\rho_{A_0B_0}, \rho_{A_1B_1}, \rho_{A_2B_2}$ 和 $\rho_{A_3B_3}$，每一份的初始态都为 $\rho_{in} = \rho_{\text{iso}}(p =0.7)$。如此，Alice 手中有四个量子比特，分别是 $A_0, A_1, A_2, A_3$；对应地，Bob 手中也有四个量子比特，$B_0, B_1, B_2, B_3$。完成上述初始化后，Alice 和 Bob 需要选择通讯的轮数（communication round）$r$，即他们需要交换经典信息（可以是各自的测量结果）多少次。为了方便讨论，这里我们选择通讯轮数为 1。接下来， Alice 和 Bob 需要进行下述操作使得 LOCCNet 学习出最优的本地操作（local operation）。更准确的说法是学习出代表本地操作的量子神经网络（quantum neural network, QNN）的最优参数。

1. 设计一个 QNN 架构 $U(\boldsymbol\theta)$ 如图 1所示，其中 $R(\theta)$ 表示单比特通用门。我们可以通过调用 Paddle Quantum 中的 `u3(theta, phi, lam, which_qubit)` 来实现该旋转门。
2. 在量子比特通过步骤 1. 中设计的 QNN 电路后，Alice 和 Bob 需要对除去 $A_0$ 和 $B_0$ 的剩余量子比特进行测量。测量结果 $M = \{m_{A_1}m_{B_1}, m_{A_2}m_{B_2}, m_{A_3}m_{B_3}\}$ 需要通过经典方式告知对方。
3. 如果每对的测量结果都相同，即 $m_{A_1}m_{B_1}, m_{A_2}m_{B_2}, m_{A_3}m_{B_3}$ 的结果要么为 00， 要么为 11。这种情况下，我们称蒸馏成功，然后剩余的一对量子比特 $A_0B_0$ 作为输出态被保存，标记为 $\rho_{AB}'$。如果测量结果不满足上述判定结果，则蒸馏失败，需要丢弃量子比特 $A_0B_0$。
4. 这里，我们在所以蒸馏成功的情况下定义一个累积的损失函数 $L = \sum_{m_{A_j}m_{B_j}\in \{00,11\}} \big(1- \text{Tr}(\rho_{tar}\rho_{AB}')\big)$，其中 $\text{Tr}(\rho_{tar}\rho_{AB}')$ 表示当前态 $\rho_{AB}'$ 和目标态 $\rho_{tar}=\lvert\Phi^+\rangle \langle \Phi^+\rvert$ 之间的态重叠。
5. 使用梯度下降的优化方案来更新 QNN 中的参数使得损失函数最小化。
6. 重复步骤 1-5 直至损失函数收敛。
7. 输出蒸馏后的态 $\rho_{out} = \rho'_{A_0B_0}$。

<center><img src="figures/distillation-fig-LOCCNet4.png" height="200" width="400"></center>
<div style="text-align:center">图 1: 由 LOCCNet 设计的纠缠蒸馏方案示意图 </div>

**注释：** 图 1中的 QNN 架构设计仅是一个示意图，并不固定。感兴趣的读者可以设计一个自己的 QNN 架构。

## Paddle Quantum 代码实现

首先，我们需要导入所有依赖包：

In [2]:
import numpy as np
import paddle
from paddle import matmul, trace
import paddle_quantum
from paddle_quantum.locc import LoccNet
from paddle_quantum.state import isotropic_state, bell_state
from paddle_quantum.qinfo import logarithmic_negativity
# 切换至密度矩阵模式
paddle_quantum.set_backend('density_matrix')

然后，我们需要定义 QNN 和 损失函数

In [3]:
class LOCC(LoccNet):
    def __init__(self):
        super(LOCC, self).__init__()

        # 添加第一个参与方 Alice
        # 第一个参数 4 代表着 Alice 手里有几个量子比特
        # 第二个参数代表着参与方的名字
        self.add_new_party(4, party_name="Alice")
        # 添加第二个参与方 Bob
        self.add_new_party(4, party_name="Bob")

        # 生成输入态 isotropic state
        _state = isotropic_state(2, 0.7)
        # 初始化量子态
        # ('Alice', 0) 代表 Alice 的第一个量子比特 A0
        # ('Bob', 0) 代表 Bob 的第一个量子比特 B0
        self.set_init_state(_state, [["Alice", 0], ["Bob", 0]])
        self.set_init_state(_state, [["Alice", 1], ["Bob", 1]])
        self.set_init_state(_state, [["Alice", 2], ["Bob", 2]])
        self.set_init_state(_state, [["Alice", 3], ["Bob", 3]])

        # 创建 Alice 的局部操作
        self.cir1 = self.create_ansatz("Alice")
        self.QNN(self.cir1)
        # 创建 Bob 的局部操作
        self.cir2 = self.create_ansatz("Bob")
        self.QNN(self.cir2)

    def QNN(self, cir):
        '''
        定义图 1 中的 QNN
        '''
        cir.u3('full')
        cir.cnot('cycle')
        cir.u3('full')

    def New_Protocol(self):
        status = self.init_status
        # 运行 Alice 的电路
        status = self.cir1(status)
        # 运行 Bob 的电路
        status = self.cir2(status)

        # 测量量子比特，["000000", "000011","001100","110000","001111","111100","110011","111111"] 代表蒸馏成功的情况
        status1 = self.measure(status, [["Alice", 1], ["Bob", 1],["Alice", 2], ["Bob", 2], ["Alice", 3], ["Bob", 3]], 
                               ["000000", "000011", "001100", "110000", "001111", "111100", "110011", "111111"])
        # 取偏迹除去测量后的量子比特，只留下 A0 & B0 
        status_fin = self.partial_state(status1, [["Alice", 0], ["Bob", 0]])
        target_state = bell_state(2)

        # 计算损失函数
        loss = 0
        for idx in range(0, len(status_fin)):
            loss += 1 - paddle.real(trace(matmul(target_state.data, status_fin[idx].data)))

        return loss, status_fin

最后，通过基于梯度的优化方法使得损失函数最小化。

In [5]:
ITR = 100    # 循环次数
LR = 0.2     # 学习率
paddle.seed(999)

net = LOCC()
# 选择 Adam 优化器
opt = paddle.optimizer.Adam(learning_rate=LR, parameters=net.cir1.parameters() + net.cir2.parameters())

# 优化循环
for itr in range(ITR):
    loss, status_fin = net.New_Protocol()
    # 反向传播
    loss.backward()
    opt.minimize(loss)
    # 清除梯度
    opt.clear_grad()
    # 打印训练结果
    if itr % 10 == 0:
        print("itr " + str(itr) + ":", loss.numpy()[0])

# 计算输入态的保真度
fidelity_in = (3 * 0.7 + 1) / 4
# 计算输出态的保真度
fidelity = (len(status_fin) - loss) / len(status_fin)
# 计算成功率
suc_rate = sum([s.prob for s in status_fin])

print("输入态的保真度为：%.5f" % fidelity_in)
print("输出态的保真度为：%.5f" % fidelity.numpy()[0])
print("提纯成功率为：%.5f" % suc_rate.numpy()[0])
rho_out = status_fin[0]
print("========================================================")
print(f"输出态为:\n {np.around(rho_out.data.numpy(), 4)}")
print(f"初始态的 logarithmic negativity 为: {logarithmic_negativity(isotropic_state(2, 0.7)).numpy()[0]}")
print(f"输出态的 logarithmic negativity 为: {logarithmic_negativity(rho_out).numpy()[0]}")

itr 0: 5.7033257
itr 10: 1.552211
itr 20: 0.8980296
itr 30: 0.7061223
itr 40: 0.57139534
itr 50: 0.5278059
itr 60: 0.5121539
itr 70: 0.50759834
itr 80: 0.5052042
itr 90: 0.50493234
输入态的保真度为：0.77500
输出态的保真度为：0.93690
提纯成功率为：0.38654
输出态为:
 [[ 4.790e-01-0.e+00j -1.000e-04-0.e+00j  1.000e-04-0.e+00j
   4.579e-01+5.e-04j]
 [-1.000e-04+0.e+00j  2.100e-02+0.e+00j  0.000e+00+0.e+00j
  -1.000e-04+0.e+00j]
 [ 1.000e-04+0.e+00j  0.000e+00-0.e+00j  2.100e-02+0.e+00j
   1.000e-04+1.e-04j]
 [ 4.579e-01-5.e-04j -1.000e-04-0.e+00j  1.000e-04-1.e-04j
   4.790e-01-0.e+00j]]
初始态的 logarithmic negativity 为: 0.6322681307792664
输出态的 logarithmic negativity 为: 0.9059638381004333


## 结论

如同数值结果所示，由 LOCCNet 设计出新型蒸馏协议可以把4份保真度为0.775的 isotropic 态蒸馏出1份保真度为0.937的两比特量子态。在相同条件下，扩展版的 DEJMPS 协议 [3] 只能够提纯出保真度为 0.924 的量子态，低于新的蒸馏协议。除了能够获得高保真度外，我们框架有着更广的适用范围和良好的可扩展性。对蒸馏的感兴趣的读者可以尝试多轮通讯会对蒸馏结果产生什么样的结果。当然，我们也欢迎读者用该框架来蒸馏含有不同噪声的态。

LOCCNet 有着广泛的应用，纠缠蒸馏仅仅是其中的一小部分。此外，我们想要强调的是通过 LOCCNet 训练出来的协议是可以在近期量子设备上实现从而进行验证的。

---

## 参考文献

[1] Chitambar, Eric, et al. "Everything you always wanted to know about LOCC (but were afraid to ask)." [Communications in Mathematical Physics 328.1 (2014): 303-326.](https://link.springer.com/article/10.1007/s00220-014-1953-9)

[2] Bennett, Charles H., et al. "Purification of noisy entanglement and faithful teleportation via noisy channels." [Physical Review Letters 76.5 (1996): 722.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.76.722)

[3] Deutsch, David, et al. "Quantum privacy amplification and the security of quantum cryptography over noisy channels." [Physical Review Letters 77.13 (1996): 2818.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.77.2818)